# Python для сбора и анализа данных
*Алла Тамбовцева, НИУ ВШЭ*

## Работа с API ВКонтакте: собираем посты со стены

Загружаем модули и библиотеки, необходимые для работы:

In [1]:
import requests
import time
import pandas as pd

Для начала давайте посмотрим на документацию API и посмотрим, как к нему формировать запросы: https://dev.vk.com/api/api-requests.

В прошлый раз по инструкции мы получили доступ к API, вспомним шаги.

In [2]:
# вводим id своего приложения
# и проходим по ссылке с этим id

app_id = input("Enter your client id: ")
url = f"https://oauth.vk.com/authorize?client_id={app_id}&display=page&redirect_uri=http://oauth.vk.com/blank.html&scope=all&response_type=token"
print(url)

Enter your client id: 7418109
https://oauth.vk.com/authorize?client_id=7418109&display=page&redirect_uri=http://oauth.vk.com/blank.html&scope=all&response_type=token


In [3]:
# копируем токен доступа

token = input("Enter your token here: ")

Enter your token here: 1543b974c5254029b759eb1d2298932d5fb691e0143074de2bd1e56edaeefe042c348015f4040195bfd16


На этом практическом занятии мы будем выгружать посты из сообщества [Цитатник ВШЭ](https://vk.com/hseteachers). Сохраним в переменные версию API, ссылку для метода работы со стеной сообщества и название сообщества:

In [4]:
v = "5.131"
main_wall = "https://api.vk.com/method/wall.get"
domain = "hseteachers"

Функция `get()` из библиотеки `requests` умеет подставлять в запрос необходимые параметры и объединять их с помощью `?` и `&`. Сохраним необходимые параметры в виде словаря:

In [5]:
params_wall = {"access_token" : token, 
              "domain" : domain, 
              "count" : 100,
              "v" : v}

А теперь сформируем запрос и выгрузим результаты в формате JSON – в Python данные в таком формате будут представлены в виде словаря:

In [6]:
req_wall = requests.get(main_wall, params = params_wall)
json_wall = req_wall.json()
json_wall

{'response': {'count': 9295,
  'items': [{'id': 36551,
    'from_id': -63442801,
    'owner_id': -63442801,
    'date': 1649157900,
    'marked_as_ads': 0,
    'post_type': 'post',
    'text': 'Это не торговля, это казино.\n\n#Володин_ВШЭ #Финансовые_рынки',
    'post_source': {'type': 'vk'},
    'comments': {'can_post': 1, 'count': 0},
    'likes': {'can_like': 1, 'count': 32, 'user_likes': 0, 'can_publish': 1},
    'reposts': {'count': 7, 'user_reposted': 0},
    'views': {'count': 1094},
    'is_favorite': False,
    'donut': {'is_donut': False},
    'short_text_rate': 0.8,
    'hash': 'TI9-Dp7TWa6U6EB4AIYWDgBoPxw'},
   {'id': 36549,
    'from_id': -63442801,
    'owner_id': -63442801,
    'date': 1649071500,
    'marked_as_ads': 0,
    'post_type': 'post',
    'text': 'Выражаясь современным политическим языком: «слежу за базаром».\n\n#Юркевич_ВШЭ #История_восточных_цивилизаций',
    'post_source': {'type': 'vk'},
    'comments': {'can_post': 1, 'count': 0},
    'likes': {'can_like'

Извлечём из этого большого словаря элемент, который хранит непосредственно результаты – список из маленьких словарей с информацией о постах (1 словарь = 1 пост):

In [7]:
items_wall = json_wall['response']['items'] 
items_wall

[{'id': 36551,
  'from_id': -63442801,
  'owner_id': -63442801,
  'date': 1649157900,
  'marked_as_ads': 0,
  'post_type': 'post',
  'text': 'Это не торговля, это казино.\n\n#Володин_ВШЭ #Финансовые_рынки',
  'post_source': {'type': 'vk'},
  'comments': {'can_post': 1, 'count': 0},
  'likes': {'can_like': 1, 'count': 32, 'user_likes': 0, 'can_publish': 1},
  'reposts': {'count': 7, 'user_reposted': 0},
  'views': {'count': 1094},
  'is_favorite': False,
  'donut': {'is_donut': False},
  'short_text_rate': 0.8,
  'hash': 'TI9-Dp7TWa6U6EB4AIYWDgBoPxw'},
 {'id': 36549,
  'from_id': -63442801,
  'owner_id': -63442801,
  'date': 1649071500,
  'marked_as_ads': 0,
  'post_type': 'post',
  'text': 'Выражаясь современным политическим языком: «слежу за базаром».\n\n#Юркевич_ВШЭ #История_восточных_цивилизаций',
  'post_source': {'type': 'vk'},
  'comments': {'can_post': 1, 'count': 0},
  'likes': {'can_like': 1, 'count': 100, 'user_likes': 0, 'can_publish': 1},
  'reposts': {'count': 10, 'user_re

Посмотрим на один элемент такого списка:

In [8]:
i = items_wall[0]
i

{'id': 36551,
 'from_id': -63442801,
 'owner_id': -63442801,
 'date': 1649157900,
 'marked_as_ads': 0,
 'post_type': 'post',
 'text': 'Это не торговля, это казино.\n\n#Володин_ВШЭ #Финансовые_рынки',
 'post_source': {'type': 'vk'},
 'comments': {'can_post': 1, 'count': 0},
 'likes': {'can_like': 1, 'count': 32, 'user_likes': 0, 'can_publish': 1},
 'reposts': {'count': 7, 'user_reposted': 0},
 'views': {'count': 1094},
 'is_favorite': False,
 'donut': {'is_donut': False},
 'short_text_rate': 0.8,
 'hash': 'TI9-Dp7TWa6U6EB4AIYWDgBoPxw'}

Поработаем с ним!

### Задача 1

Извлечь из элемента `i` следующие компоненты:

* id поста;
* дата поста;
* текст поста;
* число лайков;
* число репостов;
* число просмотров;
* число комментариев.

In [10]:
# YOUR CODE HERE
(i["id"], i["date"], i["text"], 
i["likes"]["count"], i["reposts"]["count"],
i["views"]["count"], i["comments"]["count"]) 

(36551,
 1649157900,
 'Это не торговля, это казино.\n\n#Володин_ВШЭ #Финансовые_рынки',
 32,
 7,
 1094,
 0)

### Задача 2

Напишите функцию `get_posts()`, которая принимает на вход словарь, аналогичный сохранённому в `i`, и возвращает список из следующих характеристик:

* id поста;
* дата поста;
* текст поста;
* число лайков;
* число репостов;
* число просмотров;
* число комментариев.

In [24]:
def get_posts(i):
    id_ = i["id"]
    date = i["date"]
    
    text = i.get("text")
    likes = i.get("likes").get("count")
    reposts = i.get("reposts").get("count")
    
    try:
        views = i.get("views").get("count")
    except: 
        views = None
        
    comments = i.get("comments").get("count")
    
    res = [id_, date, text, likes, reposts, views, comments] 
    return res

### Задача 3

Примените функцию `get_posts()` ко всем элементам списка `items_wall` и сохраните полученные результаты в список `posts`. 

In [12]:
posts = []
for i in items_wall:
    r = get_posts(i)
    posts.append(r) 

In [14]:
len(posts)

100

### Задача 4

Прочитайте в документации к API ВКонтакте про аргумент `offset` в методе `wall.get`. Используя полученную информацию и блоки кода ниже, выгрузите и сохраните в список `items_more` данные по всем постам на стене сообщества.

**Подсказка:** чтобы расширять список правильным образом, используйте метод `.extend()`, а не `.append()`, он добавляет не один элемент, а сразу несколько (см. примеры ниже).

In [15]:
# с append()

A = []
for i in range(5):
    B = [1, 2, 3]
    A.append(B)
print(A)

[[1, 2, 3], [1, 2, 3], [1, 2, 3], [1, 2, 3], [1, 2, 3]]


In [16]:
# с extend()

A = []
for i in range(5):
    B = [1, 2, 3]
    A.extend(B)
print(A)

[1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3]


In [17]:
params_wall_long = {"access_token" : token, 
              "domain" : domain, 
              "count" : 100,
              "offset" : 100,
              "v" : v}

In [19]:
# CHANGE CODE HERE
params_wall_long = {"access_token" : token, 
              "domain" : domain, 
              "count" : 100,
              "offset" : 100,
              "v" : v}

items_more = []  

for u in range(0, 93):
    req_wall_long = requests.get(main_wall, params = params_wall_long)
    json_wall_long = req_wall_long.json()
    items_wall_long = json_wall_long['response']['items']
    items_more.extend(items_wall_long)
    params_wall_long["offset"] += 100
    time.sleep(1) 
    print(u) 

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92


Теперь извлечём из каждого элемента `items_more` нужную информацию и расширим список `posts`, который у нас уже был до этого:

In [21]:
items_more

[{'id': 36206,
  'from_id': -63442801,
  'owner_id': -63442801,
  'date': 1644755700,
  'marked_as_ads': 0,
  'post_type': 'post',
  'text': 'Вы часто делаете оговорку все, "может быть я скажу какую-нибудь глупость” - пожалуйста, не бойтесь этого, потому что в целом мне кажется, что ваша задача, пока вы учитесь, высказывать вообще всё то, что есть у вас в голове, потому что… ну, а как?.. собственно вы сюда и пришли, для того чтобы проверять свои гипотезы о том, как устроена жизнь, разве нет?\n\n#Клюев_ВШЭ #Аудиовизуализация_в_современных_медиа',
  'post_source': {'type': 'api'},
  'comments': {'can_post': 1, 'count': 0},
  'likes': {'can_like': 1, 'count': 485, 'user_likes': 0, 'can_publish': 1},
  'reposts': {'count': 99, 'user_reposted': 0},
  'views': {'count': 17864},
  'is_favorite': False,
  'donut': {'is_donut': False},
  'short_text_rate': 0.8,
  'hash': '3hn5_ZkN49r5YODaTkymjm1ZFzE'},
 {'id': 36204,
  'from_id': -63442801,
  'owner_id': -63442801,
  'date': 1644744900,
  'mark

In [25]:
for i in items_more:
    p = get_posts(i)
    posts.append(p)

In [26]:
len(posts)  # все идёт по плану

21367

In [29]:
posts_unique = []
for p in posts:
    if p not in posts_unique:
        posts_unique.append(p)

In [30]:
len(posts_unique)

9295

Преобразуем результат в датафрейм, добавим названия столбцов:

In [31]:
dat = pd.DataFrame(posts_unique)
dat.columns = ["id", "timestamp", "post", "likes", 
               "reposts", "views", "comments"]

Несколько строк датафрейма для примера:

In [32]:
dat.head()

,id,timestamp,post,likes,reposts,views,comments
0,36551,1649157900,"Это не торговля, это казино.\n\n#Володин_ВШЭ #...",32,7,1094.0,0
1,36549,1649071500,Выражаясь современным политическим языком: «сл...,100,10,3326.0,0
2,36546,1648985100,Я Вам ещё раз повторяю: право собственности - ...,109,18,6952.0,0
3,36544,1648898700,Акбаров объясняет неопределенные интегралы\n\n...,188,37,7605.0,0
4,36542,1648812300,"*первая лекция по линейной алгебре, оглашается...",107,19,5775.0,0


Разобьём текст поста по `#`, чтобы извлечь тэги:

In [33]:
with_tags = dat["post"].str.split("#", expand = True)
with_tags

,0,1,2,3,4,5,6
0,"Это не торговля, это казино.\n\n",Володин_ВШЭ,Финансовые_рынки,None,None,None,None
1,Выражаясь современным политическим языком: «сл...,Юркевич_ВШЭ,История_восточных_цивилизаций,None,None,None,None
2,Я Вам ещё раз повторяю: право собственности - ...,Байбак_ВШЭ,юрфак,None,None,None,None
3,Акбаров объясняет неопределенные интегралы\n\n...,Акбаров_ВШЭ,Матанализ,МИЭМ,None,None,None
4,"*первая лекция по линейной алгебре, оглашается...",Поляков_ВШЭ,ФЭН,None,None,None,None
...,...,...,...,...,...,...,...
9290,по маленькой кругленькой громов\n,"Громов_hse\n_____________________\nПожалуйста,...",None,None,None,None,None
9291,Все глоки суть куздры (Данько)\n,Данько_hse,None,None,None,None,None
9292,с какого бадуна ты это написал? (Самовол)\n,Самовол_hse,None,None,None,None,None
9293,Синдром яндекса (Шаповалов И. А)\n,Шаповалов_hse,None,None,None,None,None


Основная информация – это первые два тэга, имя преподавателя и курс (по крайней мере, в большинстве случаев это так). Заберём для дальнейшей работы только их:

In [34]:
small = with_tags.loc[:, 0:2]
small.columns = ["text", "teacher", "course"] 

Склеим датафрейм `dat` с датафреймом `small` по столбцам:

In [35]:
final = pd.concat([dat, small], axis = 1)

Заполним пропуски – добавим «пустой» текст в ячейки, где нет никаких значений:

In [36]:
final = final.fillna("") 

Избавимся от лишних пробелов и отступов в текстовых данных:

In [37]:
final["text"] = final["text"].apply(lambda x: x.strip())
final["teacher"] = final["teacher"].apply(lambda x: x.strip())
final["course"] = final["course"].apply(lambda x: x.strip())

Осталось поработать с форматом времени в столбце `timestamp`.

In [38]:
final["timestamp"]

0       1649157900
1       1649071500
2       1648985100
3       1648898700
4       1648812300
           ...    
9290    1387918264
9291    1387917399
9292    1387917345
9293    1387916061
9294    1387914004
Name: timestamp, Length: 9295, dtype: int64

In [39]:
t = final["timestamp"][0]
t

1649157900

Импортируем из модуля `datetime` функцию `datetime`, она поможет нам получить дату и время в привычном формате:

In [40]:
from datetime import datetime

In [41]:
datetime.utcfromtimestamp(t) 

datetime.datetime(2022, 4, 5, 11, 25)

In [42]:
datetime.utcfromtimestamp(t).strftime('%Y-%m-%d %H:%M:%S')

'2022-04-05 11:25:00'

Напишем функцию для преобразования временной метки:

In [43]:
def time_transform(t):
    r = datetime.fromtimestamp(t).strftime('%Y-%m-%d %H:%M:%S')
    return r

Применим её ко всем элементам столбца:

In [44]:
final["datetime"] = final["timestamp"].apply(time_transform)

Теперь можем разбить дату-время по пробелу, чтобы получить отдельные столбцы с датой и временем (механизм нам уже известен, мы разбивали пост по `#` выше):

In [45]:
final

,id,timestamp,post,likes,reposts,views,comments,text,teacher,course,datetime
0,36551,1649157900,"Это не торговля, это казино.\n\n#Володин_ВШЭ #...",32,7,1094,0,"Это не торговля, это казино.",Володин_ВШЭ,Финансовые_рынки,2022-04-05 14:25:00
1,36549,1649071500,Выражаясь современным политическим языком: «сл...,100,10,3326,0,Выражаясь современным политическим языком: «сл...,Юркевич_ВШЭ,История_восточных_цивилизаций,2022-04-04 14:25:00
2,36546,1648985100,Я Вам ещё раз повторяю: право собственности - ...,109,18,6952,0,Я Вам ещё раз повторяю: право собственности - ...,Байбак_ВШЭ,юрфак,2022-04-03 14:25:00
3,36544,1648898700,Акбаров объясняет неопределенные интегралы\n\n...,188,37,7605,0,Акбаров объясняет неопределенные интегралы\n\n...,Акбаров_ВШЭ,Матанализ,2022-04-02 14:25:00
4,36542,1648812300,"*первая лекция по линейной алгебре, оглашается...",107,19,5775,0,"*первая лекция по линейной алгебре, оглашается...",Поляков_ВШЭ,ФЭН,2022-04-01 14:25:00
...,...,...,...,...,...,...,...,...,...,...,...
9290,6,1387918264,по маленькой кругленькой громов\n#Громов_hse\n...,5,0,,0,по маленькой кругленькой громов,"Громов_hse\n_____________________\nПожалуйста,...",,2013-12-25 00:51:04
9291,5,1387917399,Все глоки суть куздры (Данько)\n#Данько_hse,18,1,,2,Все глоки суть куздры (Данько),Данько_hse,,2013-12-25 00:36:39
9292,4,1387917345,с какого бадуна ты это написал? (Самовол)\n#Са...,39,2,,0,с какого бадуна ты это написал? (Самовол),Самовол_hse,,2013-12-25 00:35:45
9293,3,1387916061,Синдром яндекса (Шаповалов И. А)\n#Шаповалов_hse,23,0,,0,Синдром яндекса (Шаповалов И. А),Шаповалов_hse,,2013-12-25 00:14:21


In [46]:
dt = final["datetime"].str.split(" ", expand = True)
dt.columns = ["date", "time"]

In [47]:
final = pd.concat([final, dt], axis = 1)

In [48]:
final

,id,timestamp,post,likes,reposts,views,comments,text,teacher,course,datetime,date,time
0,36551,1649157900,"Это не торговля, это казино.\n\n#Володин_ВШЭ #...",32,7,1094,0,"Это не торговля, это казино.",Володин_ВШЭ,Финансовые_рынки,2022-04-05 14:25:00,2022-04-05,14:25:00
1,36549,1649071500,Выражаясь современным политическим языком: «сл...,100,10,3326,0,Выражаясь современным политическим языком: «сл...,Юркевич_ВШЭ,История_восточных_цивилизаций,2022-04-04 14:25:00,2022-04-04,14:25:00
2,36546,1648985100,Я Вам ещё раз повторяю: право собственности - ...,109,18,6952,0,Я Вам ещё раз повторяю: право собственности - ...,Байбак_ВШЭ,юрфак,2022-04-03 14:25:00,2022-04-03,14:25:00
3,36544,1648898700,Акбаров объясняет неопределенные интегралы\n\n...,188,37,7605,0,Акбаров объясняет неопределенные интегралы\n\n...,Акбаров_ВШЭ,Матанализ,2022-04-02 14:25:00,2022-04-02,14:25:00
4,36542,1648812300,"*первая лекция по линейной алгебре, оглашается...",107,19,5775,0,"*первая лекция по линейной алгебре, оглашается...",Поляков_ВШЭ,ФЭН,2022-04-01 14:25:00,2022-04-01,14:25:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9290,6,1387918264,по маленькой кругленькой громов\n#Громов_hse\n...,5,0,,0,по маленькой кругленькой громов,"Громов_hse\n_____________________\nПожалуйста,...",,2013-12-25 00:51:04,2013-12-25,00:51:04
9291,5,1387917399,Все глоки суть куздры (Данько)\n#Данько_hse,18,1,,2,Все глоки суть куздры (Данько),Данько_hse,,2013-12-25 00:36:39,2013-12-25,00:36:39
9292,4,1387917345,с какого бадуна ты это написал? (Самовол)\n#Са...,39,2,,0,с какого бадуна ты это написал? (Самовол),Самовол_hse,,2013-12-25 00:35:45,2013-12-25,00:35:45
9293,3,1387916061,Синдром яндекса (Шаповалов И. А)\n#Шаповалов_hse,23,0,,0,Синдром яндекса (Шаповалов И. А),Шаповалов_hse,,2013-12-25 00:14:21,2013-12-25,00:14:21
